In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/lao-airlines/?sortby=post_date%3ADesc&pagesize=100'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    try:
        for name, rating in zip(names[1:], ratings):
            review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    except:
        review_dict[name.text] = np.nan
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'lao skyway'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']


# Save to CSV
df.to_csv('Data/lao_skyway.csv',index=False)

In [3]:
# Sanity check
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,lao skyway,Gu Li,2019-12-26,✅ Trip Verified | Booked the flight to fly on...,Family Leisure,Economy Class,1,NaN,NaN,NaN,NaN,1,1,no
1,lao skyway,C Dawson,2019-11-07,✅ Trip Verified | Luangpraban to Vientiane. T...,Solo Leisure,Economy Class,9,4,5,4,NaN,5,4,yes
2,lao skyway,A Laine,2019-03-23,✅ Trip Verified | Pakse to Bangkok. Great legs...,Couple Leisure,Economy Class,10,5,5,4,3,5,5,yes
3,lao skyway,David Jackson,2019-01-26,✅ Trip Verified | Vientiane to Bangkok. UPlan...,Solo Leisure,Economy Class,10,5,5,5,NaN,5,5,yes
4,lao skyway,B Lewis,2019-01-18,✅ Trip Verified | Luang Prabang to Vientiane w...,Couple Leisure,Economy Class,8,4,3,2,NaN,3,4,yes


In [4]:
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
67,lao skyway,Andrew Leckie,2009-10-21,Luang Prabang to Siem Reap on a 70 seat turbo-...,NaN,Economy Class,9,NaN,NaN,NaN,NaN,NaN,4,yes
68,lao skyway,Vannaseng Ounalom,2009-09-14,Vientiane-Luang Prabang return. Short flight o...,NaN,Economy Class,6,NaN,NaN,NaN,NaN,NaN,4,yes
69,lao skyway,Vannaseng Ounalom,2009-09-07,Bangkok-Vientiane. The aircraft was clean a ne...,NaN,Economy Class,8,NaN,NaN,NaN,NaN,NaN,4,yes
70,lao skyway,Andrew Fleming,2009-07-04,Vientiane to Hanoi route is a code share with ...,NaN,Economy Class,7,NaN,NaN,NaN,NaN,NaN,4,yes
71,lao skyway,Paul Nesbitt,2008-12-13,Vientiane-Luang Prabang-Bangkok In November Bo...,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,no
